In [1]:
import sys
import time

# Packages for data and number handling
import numpy as np
import pandas as pd
import math

# Packages for calculating current time and extracting ZTF data to VOTable
from astropy.time import Time
from astropy.table import Table, setdiff, unique, vstack
from astropy.io.votable import parse_single_table, from_table, writeto
from datetime import datetime

# Packages for display and data plotting, if desired
from IPython.display import HTML
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
stamp_objects = parse_single_table("VOTables/ztf_api_stamp_objects.xml").to_table()
lc_objects = parse_single_table("VOTables/ztf_api_lc_objects.xml").to_table()

## LC classifier objects

In [3]:
lc_objects.info

<Table length=110105>
     name      dtype 
------------- -------
          oid   str12
     ndethist    str4
     ncovhist   int64
 mjdstarthist float64
   mjdendhist float64
    corrected    bool
      stellar    bool
         ndet   int64
      g_r_max  object
 g_r_max_corr  object
     g_r_mean  object
g_r_mean_corr  object
     firstmjd float64
      lastmjd float64
      deltajd float64
       meanra float64
      meandec float64
      sigmara float64
     sigmadec float64
        class   str14
   classifier   str13
  probability float64
 step_id_corr   str16

In [5]:
# Identify duplicate OID entries - rows with same OID but different classes and probabilities
obsgroup = lc_objects.group_by(['oid'])
duplicates = []
for key, group in zip(obsgroup.groups.keys, obsgroup.groups):
    if len(group) > 1:
        oid = group['oid'][1]
        duplicates.append(oid)

print('Number of duplicate OIDs: %i' % (len(duplicates)))

Number of duplicate OIDs: 173


In [6]:
# Print example rows with duplicate OIDs
oid = duplicates[0]
mask = (lc_objects['oid'] == oid)
lc_objects[mask]

oid,ndethist,ncovhist,mjdstarthist,mjdendhist,corrected,stellar,ndet,g_r_max,g_r_max_corr,g_r_mean,g_r_mean_corr,firstmjd,lastmjd,deltajd,meanra,meandec,sigmara,sigmadec,class,classifier,probability,step_id_corr
str12,str4,int64,float64,float64,bool,bool,int64,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,str14,str13,float64,str16
ZTF17aaaecgj,453,915,58095.136099500116,59329.48268519994,True,True,388,0.7726,0.940893,0.97359747,0.8993047,58285.42535880022,59329.48268519994,1044.0573263997212,314.6341218953608,56.741510996907216,0.0001113494073664217,8.862299233714988e-05,E,lc_classifier,0.22878,correction_0.0.1
ZTF17aaaecgj,453,915,58095.136099500116,59329.48268519994,True,True,388,0.7726,0.940893,0.97359747,0.8993047,58285.42535880022,59329.48268519994,1044.0573263997212,314.6341218953608,56.741510996907216,0.0001113494073664217,8.862299233714988e-05,RRL,lc_classifier,0.22878,correction_0.0.1


New objects that are detected and classified by the LC classifier:

In [7]:
# First, remove duplicate OID rows
lc_objects_unique = unique(lc_objects, keys='oid')

new_lc_objects = lc_objects_unique[(lc_objects_unique['ndet'] <= 6)]

print("Number of newly detected objects : %i" % len(new_lc_objects))
print(f"Classes of newly detected objects : {sorted(set(new_lc_objects['class'].data))}")
print(f"Ratio of new- to previously- detected objects : {len(new_lc_objects)/(len(lc_objects_unique)-len(new_lc_objects))}")

Number of newly detected objects : 178
Classes of newly detected objects : ['AGN', 'CEP', 'CV/Nova', 'DSCT', 'LPV', 'Periodic-Other', 'RRL', 'SNIbc', 'YSO']
Ratio of new- to previously- detected objects : 0.001621808772345427


## Stamp classifier objects

In [8]:
stamp_objects.info

<Table length=149117>
     name      dtype 
------------- -------
          oid   str12
     ndethist    str4
     ncovhist   int64
 mjdstarthist float64
   mjdendhist float64
    corrected    bool
      stellar    bool
         ndet   int64
      g_r_max  object
 g_r_max_corr  object
     g_r_mean  object
g_r_mean_corr  object
     firstmjd float64
      lastmjd float64
      deltajd float64
       meanra float64
      meandec float64
      sigmara  object
     sigmadec  object
        class    str8
   classifier   str16
  probability float64
 step_id_corr   str16

In [9]:
# Identify duplicate OID entries - rows with same OID but different classes and probabilities
obsgroup = stamp_objects.group_by(['oid'])
duplicates = []
for key, group in zip(obsgroup.groups.keys, obsgroup.groups):
    if len(group) > 1:
        oid = group['oid'][1]
        duplicates.append(oid)

print('Number of duplicate OIDs: %i' % (len(duplicates)))
# Print example rows with duplicate OIDs
if len(duplicates) > 0:
    oid = duplicates[0]
    mask = (lc_objects['oid'] == oid)
    lc_objects[mask]

Number of duplicate OIDs: 0


New objects that are detected and classified by the stamp classifier:

In [10]:
# First, remove duplicate OID rows
stamp_objects_unique = unique(stamp_objects, keys='oid')

new_stamp_objects = stamp_objects_unique[(stamp_objects_unique['ndet'] <= 1)]

print("Number of newly detected objects : %i" % len(new_stamp_objects))
print(f"Classes of newly detected objects : {sorted(set(new_stamp_objects['class'].data))}")
print(f"Ratio of new- to previously- detected objects : {len(new_stamp_objects)/(len(stamp_objects_unique)-len(new_stamp_objects))}")

Number of newly detected objects : 20703
Classes of newly detected objects : ['AGN', 'SN', 'VS', 'asteroid', 'bogus']
Ratio of new- to previously- detected objects : 0.1612207391717414
